In [ ]:
import os
from datetime import datetime
from typing import Literal

import torch
import wandb
import numpy as np
import tensorflow as tf
import tensorflow_ranking as tfr
from torch import nn
from torch import optim
import torch_geometric.nn as geonn
from tqdm import tqdm

from gfos.data.utils import load_layout
from gfos.data.dataset import LayoutDataset
from gfos.utils.scheduler import CosineAnnealingWarmupRestarts
from gfos.metrics import kendall, topk_error
from gfos.loss import MultiElementRankLoss
from gfos.utils.misc import seed_everything


SEED = 42
DEBUG = False

seed_everything(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"

os.environ["WANDB_NOTEBOOK_NAME"] = "layout_gnn_tfm_1.ipynb"


## Configs

In [ ]:
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"

configs = dict(
    conv_layer="GATConv",
    num_epochs=8000,
    learning_rate_slow=1e-3,
    learning_rate_fast=2e-3,
    weight_decay=0.0,
    min_lr=1e-7,
    warmup_ratio=0.0,
    max_configs=4000,
    num_configs=16,
    graph_hidden=[64, 64, 64, 128, 128],
    num_encoder=4,
    num_feedforward=512,
    nhead=2,
    loss_margin=1.5,
    loss_num_permutations=50,
)

WANDB_PROJECT = "gfos"
WANDB_DIR = "../../logs/"
WANDB_RUN_NAME = "layout_xla_default"
TAGS = ["train", "layout", "xla", "default"]

NUM_VAL_EPOCHS = 100
INFERENCE_CONFIGS_BATCH_SIZE = 50
TOP_K = 500

## Model

In [ ]:
def aggregate_neighbors(node_feat: torch.Tensor, edge_index: torch.Tensor):
    source_nodes = edge_index[0]
    target_nodes = edge_index[1]

    in_degree_features = torch.zeros_like(node_feat, device=node_feat.device)
    out_degree_features = torch.zeros_like(node_feat, device=node_feat.device)

    source_node_features = node_feat[source_nodes]
    target_node_features = node_feat[target_nodes]

    in_degree_features.scatter_reduce_(
        0,
        target_nodes.unsqueeze(-1).expand_as(source_node_features),
        source_node_features,
        reduce="mean",
        include_self=False,
    )

    out_degree_features.scatter_reduce_(
        0,
        source_nodes.unsqueeze(-1).expand_as(target_node_features),
        target_node_features,
        reduce="mean",
        include_self=False,
    )

    return out_degree_features - in_degree_features

In [ ]:
class LayoutModel(torch.nn.Module):
    def __init__(
        self,
        conv_layer: Literal["GATConv", "GCNConv", "SAGEConv"],
        hidden_channels: list[int],
        num_encoder: int = 1,
        num_feedforward: int = 256,
        nhead: int = 1,
    ):
        super().__init__()

        conv_layer = getattr(geonn, conv_layer)

        op_embedding_dim = 32
        config_dim = 64
        graph_out = hidden_channels[-1]
        merged_node_dim = graph_out + config_dim

        self.embedding = torch.nn.Embedding(
            120,
            op_embedding_dim,
        )
        assert len(hidden_channels) > 0
        in_channels = op_embedding_dim + 140
        # self.convs = torch.nn.ModuleList()

        # hidden_channels = [in_channels] + hidden_channels
        # for in_channel, out_channel in zip(
        #     hidden_channels[:-1], hidden_channels[1:]
        # ):
        #     self.convs += [
        #         conv_layer(in_channel, out_channel, dropout=0.1),
        #         nn.LeakyReLU(),
        #     ]
        self.convs = geonn.Sequential('x, edge_index', [
            (nn.Dropout(p=0.1), 'x -> x'),

            (conv_layer(in_channels, hidden_channels[0]), 'x, edge_index -> x1'),
            nn.LeakyReLU(inplace=True),
            (conv_layer(hidden_channels[0], hidden_channels[1]), 'x1, edge_index -> x2'),
            nn.LeakyReLU(inplace=True),
            # (lambda x1, x2: x1 + x2, 'x1, x2 -> x3'),

            (conv_layer(hidden_channels[1], hidden_channels[2]), 'x2, edge_index -> x3'),
            nn.LeakyReLU(inplace=True),
            (lambda x4, x5: x4 + x5, 'x1, x3 -> x4'),
            (conv_layer(hidden_channels[2], hidden_channels[3]), 'x4, edge_index -> x5'),
            nn.LeakyReLU(inplace=True),
            (conv_layer(hidden_channels[3], hidden_channels[3]), 'x5, edge_index -> x6'),
            nn.LeakyReLU(inplace=True),
            (conv_layer(hidden_channels[3], hidden_channels[4]), 'x6, edge_index -> x7'),
            nn.LeakyReLU(inplace=True),
            (lambda x4, x5: x4 + x5, 'x5, x7 -> x8'),
        ])

        # Transformer encoder to merge configs features with graph features
        layer = nn.TransformerEncoderLayer(
            d_model=merged_node_dim,
            dim_feedforward=num_feedforward,
            nhead=nhead,
            batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(layer, num_layers=num_encoder)

        self.config_prj = nn.Sequential(
            nn.Linear(18, config_dim),
            nn.LayerNorm(config_dim),
            nn.LeakyReLU(),
        )

        self.deg_prj = nn.Sequential(
            nn.Linear(hidden_channels[-1], merged_node_dim, bias=False),
            nn.LayerNorm(merged_node_dim),
            nn.LeakyReLU(),
        )

        # Define a sequential dense neural network
        self.dense = torch.nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(merged_node_dim, 64, bias=False),
            nn.LeakyReLU(),
            nn.Linear(64, 64, bias=False),
            nn.LeakyReLU(),
            nn.Linear(64, 1, bias=False),
        )

    def forward(
        self,
        node_feat: torch.Tensor,
        node_opcode: torch.Tensor,
        edge_index: torch.Tensor,
        node_config_feat: torch.Tensor,
        node_config_ids: torch.Tensor,
    ) -> torch.Tensor:
        # Get graph features
        c = node_config_feat.size(0)

        x = torch.cat([node_feat, self.embedding(node_opcode)], dim=1)

        # Get graph features
        # for conv in self.convs:
        #     x = conv(x, edge_index)
        x = self.convs(x, edge_index)

        neighbor_feat = aggregate_neighbors(x, edge_index)[node_config_ids]

        # (N, graph_out) -> (NC, graph_out)
        x = x[node_config_ids]
        # x += neighbor_feat

        # Merge graph features with config features
        # (C, NC, 18) -> (C, NC, config_dim)
        node_config_feat = self.config_prj(node_config_feat)
        pos_embedding = self.deg_prj(neighbor_feat)

        # (C, NC, graph_out + config_dim)
        x = torch.cat([x.repeat((c, 1, 1)), node_config_feat], dim=-1)
        x += pos_embedding
        x = nn.functional.normalize(x, dim=-1)

        # (C, NC, graph_out + config_dim) -> (C, graph_out + config_dim)
        x = self.encoder(x)[:, -1, :]
        x = self.dense(x).flatten()

        return x

## Prepare for training

### Expand configs

In [ ]:
conv_layer = configs["conv_layer"]
num_epochs = configs["num_epochs"]
learning_rate_slow = configs["learning_rate_slow"]
learning_rate_fast = configs["learning_rate_fast"]
weight_decay = configs["weight_decay"]
min_lr = configs["min_lr"]
warmup_ratio = configs["warmup_ratio"]
max_configs = configs["max_configs"]
num_configs = configs["num_configs"]
graph_hidden = configs["graph_hidden"]
num_encoder = configs["num_encoder"]
num_feedforward = configs["num_feedforward"]
nhead = configs["nhead"]
margin = configs["loss_margin"]
number_permutations = configs["loss_num_permutations"]


### Model, loss, optimizer, scheduler

In [ ]:
model = LayoutModel(
    conv_layer=conv_layer,
    hidden_channels=graph_hidden,
    num_encoder=num_encoder,
    num_feedforward=num_feedforward,
    nhead=nhead,
).to(device)

criterion = MultiElementRankLoss(
    margin=margin, number_permutations=number_permutations
)

optimizer = optim.Adam([
    {"name": "lr_embed", 'params': model.embedding.parameters(), 'lr': learning_rate_slow},
    {"name": "lr_config", 'params': model.config_prj.parameters(), 'lr': learning_rate_slow},
    {"name": "lr_gnn", 'params': model.convs.parameters(), 'lr': learning_rate_slow},
    {"name": "lr_deg", 'params': model.deg_prj.parameters(), 'lr': learning_rate_slow},
    {"name": "lr_encoder", 'params': model.encoder.parameters(), 'lr': learning_rate_fast},
    {"name": "lr_dense", 'params': model.dense.parameters(), 'lr': learning_rate_fast},
])

# Read data
layout_data = load_layout(
    LAYOUT_DIR,
    model_type="xla",
    compile_type="random",
)

num_steps = len(layout_data["train"]) * num_epochs
warmup_steps = int(num_steps * warmup_ratio)

# scheduler = CosineAnnealingWarmupRestarts(
#     optimizer=optimizer,
#     first_cycle_steps=num_steps,
#     min_lr=min_lr,
#     max_lr=learning_rate,
#     warmup_steps=warmup_steps,
# )

scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=num_steps,
    eta_min=min_lr,
)

In [ ]:
tf.config.set_visible_devices([], 'GPU')

# Don't know why but this cost huge amount of VRAM
opa = tfr.keras.metrics.OPAMetric()


### Init wandb and train&valid dataset

In [ ]:
if not DEBUG:
    run = wandb.init(
        project=WANDB_PROJECT,
        dir=WANDB_DIR,
        name=WANDB_RUN_NAME,
        config=configs,
        tags=TAGS,
    )
    run.watch(model, log="all")
    run.log_code("../")

    time_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_dir = f"../../logs/{WANDB_RUN_NAME}/{time_str}"
    os.makedirs(log_dir, exist_ok=True)

best_score = -1

train_dataset = LayoutDataset(
    layout_data["train"], max_configs=max_configs, num_configs=num_configs,
)
val_dataset = LayoutDataset(layout_data["valid"])


## Training

In [ ]:
accum_iter = 8

for epoch in range(num_epochs):
    # Shuffle the training dataset
    permutation = np.random.permutation(len(train_dataset))

    # Training phase
    model.train()
    pbar = tqdm(permutation, leave=False)

    for i in pbar:
        record = train_dataset[i]
        node_feat = record["node_feat"]
        node_opcode = record["node_opcode"]
        edge_index = record["edge_index"]
        node_config_feat = record["node_config_feat"]
        node_config_ids = record["node_config_ids"]
        config_runtime = record["config_runtime"]

        (
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
            config_runtime,
        ) = (
            node_feat.to(device),
            node_opcode.to(device),
            edge_index.to(device),
            node_config_feat.to(device),
            node_config_ids.to(device),
            config_runtime.to(device),
        )

        out = model(
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
        )

        loss = criterion(out, config_runtime)
        # loss = listMLE(out[None], config_runtime[None])
        
        loss = loss / accum_iter 
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

        if ((i + 1) % accum_iter == 0) or (i + 1 == len(train_dataset)):
            optimizer.step()
            optimizer.zero_grad()

            if not DEBUG:
                wandb.log(
                    {
                        "epoch": epoch,
                        "train/lr_slow": scheduler.get_last_lr()[0],
                        "train/lr_fast": scheduler.get_last_lr()[-1],
                        "train/loss": loss.item(),
                    }
                )
        scheduler.step()

        pbar.set_description(f"epoch: {epoch} loss: {(loss.item()):.4f}")

    pbar.close()

    if (epoch + 1) % NUM_VAL_EPOCHS != 0 and epoch != num_epochs - 1:
        continue

    model.eval()

    # Validation phase
    # Scores placeholder
    kendalltau_scores = []
    opa_scores = []
    topk_scores = []

    with torch.no_grad():
        for record in tqdm(val_dataset, desc="valid", leave=False):
            node_feat = record["node_feat"]
            node_opcode = record["node_opcode"]
            edge_index = record["edge_index"]
            node_config_feat = record["node_config_feat"]
            node_config_ids = record["node_config_ids"]
            config_runtime = record["config_runtime"]

            (
                node_feat,
                node_opcode,
                edge_index,
                node_config_feat,
                node_config_ids,
            ) = (
                node_feat.to(device),
                node_opcode.to(device),
                edge_index.to(device),
                node_config_feat.to(device),
                node_config_ids.to(device),
            )
            config_runtime = config_runtime.numpy()
            num_configs = config_runtime.shape[-1]
            outs = []

            for i in range(0, num_configs, INFERENCE_CONFIGS_BATCH_SIZE):
                end_i = min(i + INFERENCE_CONFIGS_BATCH_SIZE, num_configs)
                out: torch.Tensor = model(
                    node_feat,
                    node_opcode,
                    edge_index,
                    node_config_feat[i:end_i],
                    node_config_ids,
                )
                outs.append(out.detach().cpu())

            outs = torch.concat(outs).numpy()

            kendalltau_scores.append(kendall(outs, config_runtime))
            opa_scores.append(opa(config_runtime[None], outs[None]))
            topk_scores.append(topk_error(outs, config_runtime, top_k=TOP_K))

    kendalltau_mean = np.mean(kendalltau_scores)
    opa_mean = np.mean(opa_scores)
    topk_mean = np.mean(topk_scores)

    if not DEBUG:
        wandb.log(
            {
                "val/kendalltau": kendalltau_mean,
                "val/opa": opa_mean,
                "val/top500_error": topk_mean,
            }
        )

    print(
        f"epoch {epoch}, kendall = {kendalltau_mean:.4f},"
        f"opa = {opa_mean:.4f}, "
        f"topk = {topk_mean:.4f}"
    )

    # Update best scores and save the model if the mean score improves
    if kendalltau_mean > best_score:
        best_score = kendalltau_mean
        print(f"Best score updated: {best_score:.4f}")
        if not DEBUG:
            filename = f"{epoch}_{kendalltau_mean:.4f}.pth"
            path = os.path.join(wandb.run.dir, filename)
            torch.save(
                model.state_dict(),
                path,
            )

if not DEBUG:
    run.finish()